### Euler's Totient Function (ETF) and GCD sum

- Problem: Can we find $x = \sum_{i=1}^{N} \text{GCD}(i, N)$ for a given value of $N$?

- Brute force: Loop from 1 to N, adding GCD of $(i, N)$ for each iteration. 
    - Recall that $GCD$ can be computed using Euclid's algorithm, which has $O(\log(N))$ time complexity
    - We have $N$ iterations, so the `gcd()` function is called $N$ times
    - This gives us a total of $O(N \log(N))$ for each query input $N$

- We will explore how we can solve this in $O(\sqrt{N})$ time!

- Some observations first
    - Imagine $N=12$
    - Note 1: `gcd(i, N)` must be a divisor of $N$. 
        - Recall that we can get all the divisors of $N$ in $O(\sqrt{N})$ time (looping over 1 to $\sqrt{N}$ to find candidate divisors)
    - Note 2: 
        - Since $N=12$, we want go find $S = \{\text{gcd}(1,12), \text{gcd}(2,12), ... \text{gcd}(12,12)\}$
        - In the brute force way, we loop from 1 to 12 and compute the `gcd` function
        - Let's make use of **Note 1**; that the `gcd` must return a divisor of 12
            - So instead of looping from 1 to 12, we can instead loop over the divisors of 12, which can be attained in $O(\sqrt{N})$ time!
            - But there is a catch, since we know the value of the divisors, we now need a way to know how many times they appear!
                - For example, 1 is a divisor of 4 items in $S$; $\text{gcd}(1,12), \text{gcd}(5,12), \text{gcd}(7,12), \text{gcd}(11,12)$
                - So the contribution of 1 to $x = \sum_{i=1}^{N} \text{GCD}(i, N)$ is $4 * 1$
        
- Let's write some pseudo template code based on the observations above. For now, we assume we have a `get_count()` function that allows us to work through this in $O(\sqrt{N})$ time
    - Notice that `get_count` has to be constant time, for the `gcd_sum` function to be $O(\sqrt{N})$ time complexity

In [1]:
def gcd_sum(N):
    result = 0
    for i in range(1,N+1):
        if N % i == 0:
            # i divides n, so find the divisor pair (i,j) where j=N/i
            divisor1 = i
            divisor2 = N/i
            result += i * get_count(divisor1, N)
            
            if divisor1 != divisor2:
                result += i * get_count(divisor2, N)
    return result

### Evaluating `get_count` with Euler's Totient Function

- Problem: Count the number of integers $i$ from 1 to N where $\text{gcd}(i, N) = d$
    - Let's assume there are integers $x_1 ... x_m$ between 1 and N that meets the criteria that $\text{gcd}(x_i, N) = d$
    - If $\text{gcd}(x_i, N) = d$, then $\text{gcd}(\frac{x_i}{d}, \frac{N}{d}) = 1$
        - Why? Because if your write $x_i, N$ as a product of prime factors, dividing by their GCD simply means removing all the primes they have in common! That must mean that they are now co-prime after division by GCD
    - That must mean that $x_i$ where $\text{gcd}(x_i, N) = d$ forms a bijection (1:1 mapping) with all numbers meeting $\text{gcd}(\frac{x_i}{d}, \frac{N}{d}) = 1$
        - But $\text{gcd}(\frac{x_i}{d}, \frac{N}{d}) = 1$ is just akin to finding all numbers from 1 to N that are co-prime with $N$!!!
        - This is simply Euler's Totient Function $\phi(N)$
        - That is; $\text{count}(x_i, \text{where } \text{gcd}(x_i, N) = d) = \phi(N)$

    - So to perform `get_count` in constant time, we just pre-compute $\phi(X)$ for all values of $X$ between 1 and some large number $N$, then look it up in constant time

In [9]:
# list(range(4, 100, 2))
N = 100

In [54]:
def erastothenes_sieve(n):
    '''
    O(N log log N)
    '''
    sieve = [-1] * (n+1)
    sieve[0], sieve[1] = 0, 1
    for i in range(2, n+1):
        sieve[i**i: n+1: i] = [int(i)] * len(sieve[i**i: n+1: i])
    return sieve

def get_prime_factors(n):
    '''
    O(log N)
    '''
    prime_factors = []
    while n != 1:
        curr_prime = sieve[n]
        if sieve[n] == -1:
            prime_factors.append(n)
            n = 1
            continue
        n = int(n / sieve[n])
        prime_factors.append(int(sieve[n]))
    return set(prime_factors)

def eulers_totient_function(n):
    '''
    O(N log log N) for sieve computation
    '''
    if n == 0:
        return 0
    result = n
    prime_factors = get_prime_factors(n)
    for prime in prime_factors:
        result *= ((prime-1)/prime)
    return result

def get_count(divisor, N):
    return phi[int(N/divisor)]

def gcd_sum(N):
    result = 0
    for i in range(1,int(N**0.5)+1):
        # print(f"{N=}, {i=}, {N%i=}")
        if N % i == 0:
            # i divides n, so find the divisor pair (i,j) where j=N/i
            divisor1 = i
            divisor2 = N/i
            result += (i * get_count(divisor1, N))
            if divisor1 != divisor2:
                result += (i * get_count(divisor2, N))
    return result

In [57]:
gcd_sum(N)

663.0